# CoNaLA

In [3]:
import string
import re


QUOTED_STRING_RE = re.compile(
    r"(?P<quote>['`\"])(?P<string>.*?)(?<!\\)(?P=quote)")


def parse_quoted_string(search_string):
    return QUOTED_STRING_RE.findall(search_string)

def parameterized(s, t, fmt='_{}_', A=string.ascii_uppercase):
    quotes = parse_quoted_string(s)
    t = f' {t} '
    index = 0
    d = {}
    dt = {}
    for q in quotes:
        qq = f'{q[0]}{q[1]}{q[0]}'
        var = fmt.format(A[len(d)])
        if qq in t:
            d[var] = qq
            dt[var] = qq
            t = t.replace(qq, var)
            s = s.replace(qq, var)
            continue
        if q[1] in t:
            d[var] = qq
            dt[var] = q[1]
            if q[1].isalnum():
                pat = f'([^A-Za-z0-9])({q[1]})([^A-Za-z0-9])'
                t = re.sub(pat, f'\\1{var}\\3', t)
            else:
                t = t.replace(q[1], var)
            s = s.replace(qq, var)
    return s, t.strip(), d, dt


In [11]:
import csv
lines = []
with open('conala5_train.tsv') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        s, t, _, _ = parameterized(row[0], row[1])
        lines.append((s, t))
with open('conala_train.py', 'w') as w:
    for s, t in lines:
        print(t, file=w)
        print(f"'''\n{s}\n'''\n", file=w)